## Setup Environment

In [1]:
!pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cpu


In [2]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
device = torch.device('mps')

In [4]:
device

device(type='mps')

In [5]:
! pip install --upgrade pip
! pip install transformers[torch]
! pip install tokenizers
! pip install --upgrade transformers
! pip install datasets
! pip install rouge
! pip install --upgrade datasets

zsh:1: no matches found: transformers[torch]


In [9]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import EncoderDecoderModel, BertTokenizer
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from rouge import Rouge
from tqdm import tqdm
import os
import json

## Load Data

- Canonical Data

In [10]:
def load_json_files(directory):
    data = {}
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as json_file:
                    data[file] = json.load(json_file)
    return data

dev_path = "./liputan6_data/canonical/dev/"
test_path = "./liputan6_data/canonical/test/"
train_path = "./liputan6_data/canonical/train/"

data_canonical_dev = load_json_files(dev_path)
data_canonical_test = load_json_files(test_path)
data_canonical_train = load_json_files(train_path)

## Change Canonical Data to DataFrame

In [11]:
pd.set_option('display.max_colwidth', None)

- Canonical Dev Dataframe

In [12]:
df_canonical_dev = pd.DataFrame.from_dict(data_canonical_dev, orient="index")
df_canonical_dev = df_canonical_dev.drop(columns=['url', 'id'])
df_canonical_dev.head()

,clean_article,clean_summary,extractive_summary
3721.json,"[[Liputan6, ., com, ,, Jakarta, :, Keinginan, untuk, menindaklanjuti, hasil, curah, pendapat, yang, diprakarsai, Kwik, Kian, Gie, ,, semakin, mengental, .], [Buktinya, ,, beberapa, anggota, DPR, ngebet, membentuk, Forum, Lobi, .], [Ketua, DPR, Akbar, Tandjung, baru-baru, ini, ,, mempersilakan, para, wakil, rakyat, yang, ingin, membentuk, forum, tersebut, .], [Namun, ,, Akbar, menegaskan, ,, pembentukan, forum, tersebut, bukan, bagian, dari, mekanisme, DPR, .], [Menurut, Akbar, ,, hasil, curah, pendapat, 11, November, silam, ,, bisa, saja, dimasukkan, dalam, agenda, dewan, .], [Asalkan, ,, kata, Ketua, Umum, Golkar, ini, ,, semua, proses, harus, berjalan, sesuai, prosedur, yang, ada, .], [Artinya, ,, itu, harus, disepakati, melalui, rapat, fraksi, dan, dibawa, ke, Badan, Musyawarah, .], ["", Bila, Bamus, telah, merumuskannya, menjadi, sebuah, usulan, agenda, ,, maka, dapat, dibawa, ke, rapat, paripurna, yang, nantinya, akan, menjadi, agenda, DPR, ,, "", kata, Akbar, ., (, YYT/Diah, Kusuma, dan, Prihandoyo, ), .]]","[[Sebagian, anggota, DPR, berniat, besar, menindaklanjuti, hasil, curah, pendapat, menjadi, agenda, dewan, .], [Akbar, Tandjung, setuju, membentuk, Forum, Lobi, ,, asal, ,, sesuai, prosedur, yang, ada, di, DPR, .]]","[1, 4]"
2833.json,"[[Liputan6, ., com, ,, Jakarta, :, Gubernur, DKI, Jakarta, Sutiyoso, mengakui, ,, pendapatan, pajak, dari, kendaraan, bermotor, khususnya, mobil, mewah, tidak, didapat, dengan, optimal, .], [Guna, meningkatkan, Pendapatan, Asli, Daerah, DKI, Jakarta, ,, sutiyoso, meminta, Kepala, Dinas, Pendapatan, Daerah, menertibkan, penerimaan, pajak, mobil, mewah, yang, selama, ini, dinilai, tak, transparan, .], [Penegasan, ini, disampaikannya, seusai, meresmikan, Gedung, Kantor, Bersama, Samsat, Unit, Pelayanan, Pajak, Kendaraan, Bermotor, dan, Bea, Balik, Nama, Kendaraan, Bermotor, di, Jakarta, Barat, ,, Jumat, (, 27/10, ), .], [Pajak, mobil, mewah, ,, kata, Sutiyoso, ,, seharusnya, dapat, dioptimalkan, seiring, dengan, banyaknya, mobil, mewah, yang, masuk, ke, Indonesia, .], [Selama, ini, pemilik, mobil, mewah, membayar, pajaknya, melalui, oknum, dengan, biaya, lebih, murah, .], [Karena, itu, ,, Pemerintah, Daerah, Jakarta, akan, bekerja, sama, dengan, Kepolisian, Daerah, Metro, Jaya, untuk, menangani, sejumlah, oknum, yang, membiarkan, pemilik, mobil, mewah, tidak, membayar, pajak, .], [Menurut, Sutiyoso, ,, pajak, kendaraan, bermotor, adalah, andalan, utama, PAD, DKI, Jakarta, .], [Pemilik, mobil, mewah, sudah, sepantasnya, dikenakan, pajak, lebih, mahal, karena, mereka, tergolong, orang, kaya, ., (, ULF/Insan, Kamil, dan, Satya, Pandia, ), .]]","[[Pendapatan, Asli, Daerah, Jakarta, kerap, terkumpul, secara, tak, optimal, .], [Penyebabnya, ,, banyak, pemilik, mobil, mewah, yang, membayar, pajak, lewat, calo, .]]","[1, 4]"
729.json,"[[Liputan6, ., com, ,, Jakarta, :, Sejumlah, mobil, yang, terjebak, di, ruang, basement, Gedung, Bursa, Efek, Jakarta, akibat, ledakan, bom, di, lantai, parkir, ,, Jumat, (, 15/9, ), pagi, ,, mulai, diambil, pemiliknya, .], [Sebagian, mobil, lainnya, belum, bisa, dikeluarkan, karena, masih, harus, diperiksa, tim, dari, Pusat, Laboratorium, Forensik, Kepolisian, Republik, Indonesia, .], [Dari, pantauan, SCTV, ,, sekitar, pukul, 10, ., 00, WIB, ,, jumlah, mobil, yang, sudah, dikeluarkan, mencapai, 45, unit, .], [Untuk, mengambil, mobil, tersebut, ,, para, pemilik, harus, menunjukkan, bukti, kepemilikan, ,, seperti, Surat, Tanda, Nomor, Kendaraan, Bermotor, ,, Buku, Pemilik, Kendaraan, Bermotor, ,, dan, karcis, parkir, .], [Mobil, yang, boleh, diambil, adalah, mobil, yang, berada, di, luar, garis, polisi, .], [Sedangkan, mobil, yang, berada, di, dalam, garis, polisi, belum, bisa, diambil, seraya, menunggu, izin, dari, Tim, Puslabfor, Polri, .], [Hal, itu, dilakukan, agar, penyelidikan, polisi, tak, mengalami, gangguan, .], [Menurut, petugas, parkir, ,, jumlah, total, mobil, di, lantai, P2, saat, terjadi, ledaka

- Canonical Test Dataframe

In [13]:
df_canonical_test = pd.DataFrame.from_dict(data_canonical_test, orient="index")
df_canonical_test = df_canonical_test.drop(columns=['url', 'id'])
df_canonical_test.head()

,clean_article,clean_summary,extractive_summary
23684.json,"[[Liputan6, ., com, ,, Bangka, :, Kapal, patroli, Angkatan, Laut, Republik, Indonesia, ,, Belinyu, ,, baru-baru, ini, ,, menangkap, tiga, kapal, nelayan, berbendera, Thailand, ,, yakni, KM, Binatama, ,, KM, Sumber, Jaya, II, ,, dan, KM, Mataram, di, Perairan, Belitung, Utara, .], [Ketiga, kapal, itu, ditangkap, karena, melanggar, zona, ekonomi, ekslusif, Indonesia, .], [Saat, ini, ,, kapal-kapal, itu, diamankan, di, Pos, Lanal, Pelabuhan, Pangkalan, Balam, ,, Bangka-Belitung, .], [Menurut, Komandan, Pangkalan, TNI, AL, Bangka, Letnan, Kolonel, Laut, Fredy, Egam, ,, selain, menangkap, tiga, kapal, ,, ALRI, juga, memeriksa, 43, anak, buah, kapal, .], [Mereka, disergap, saat, sedang, mengangkat, jaring, pukat, harimau, di, Perairan, Belitung, Utara, .], [Dari, jumlah, itu, ,, hanya, enam, orang, yang, dijadikan, tersangka, ,, yakni, tiga, nahkoda, dan, tiga, kepala, kamar, mesin, kapal, .], [Sedangkan, ABK, yang, lain, akan, dideportasi, ke, negara, asalnya, .], [Meski, berhasil, menahan, enam, tersangka, ,, TNI, AL, gagal, mengamankan, ikan, tangkapan, nelayan, Thailand, tersebut, .], [Sebab, ,, sebelum, patroli, datang, ,, mereka, telah, memindahkan, puluhan, ton, ikan, hasil, jaringan, ke, kapal, induk, ., (, ULF/Ajmal, Rokian, dan, Yanuar, Ichrom, ), .]]","[[Meski, memiliki, izin, resmi, ,, TNI, AL, tetap, menangkap, tiga, kapan, nelayan, berbendera, Thailand, .], [Pasalnya, ,, ketiga, kapal, itu, melanggar, zona, ekonomi, ekslusif, dan, menjaring, ikan, dengan, pukat, harimau, .]]","[1, 7]"
24141.json,"[[Liputan6, ., com, ,, Bandar, Lampung, :, Sebanyak, 51, anak, di, bawah, umur, lima, tahun, terserang, busung, lapar, atau, marasmus, karena, kekurangan, gizi, di, Kota, Madya, Bandar, Lampung, .], [Lima, di, antaranya, tewas, .], [Data, tersebut, diungkapkan, Kepala, Dinas, Kesehatan, Kota, Bandar, Lampung, M, .], [Sudarman, ,, baru-baru, ini, .], [Menurut, Sudarman, ,, Dinas, Kesehatan, Bandar, Lampung, mencatat, sekitar, 51, anak, terserang, busung, lapar, yang, tersebar, di, beberapa, kecamatan, ,, selama, periode, 1999, sampai, 2001, .], [Kebanyakan, anak, penderita, busung, tersebut, berasal, dari, keluarga, yang, hidup, di, bawah, garis, kemiskinan, .], [Selain, kekurangan, gizi, ,, komplikasi, radang, paru-paru, juga, menjadi, satu, faktor, penyebab, kematian, anak, penderita, busung, lapar, tersebut, .], [Data, Dinas, Kesehatan, menunjukkan, pada, 1999, ,, ditemukan, 41, anak, terserang, penyakit, busung, lapar, .], [Sebagian, besar, penderita, berdomisili, di, kampung, miskin, Umbul, Kunci, .], [Jumlah, penderita, busung, lapar, menurun, pada, 2000, ,, yakni, hanya, sembilan, anak, .], [Sedangkan, September, 2001, ,, seorang, anak, meninggal, karena, marasmus, .], [Sudarman, menegaskan, ,, untuk, menekan, jumlah, korban, marasmus, ,, anak-anak, dan, balita, diberi, makanan, tambahan, ke, sentra-sentra, rawan, busung, lapar, seperti, di, Desa, Umbul, Unci, .], [Tetapi, ,, berdasarkan, keterangan, masyarakat, Umbul, Kunci, ,, program, makanan, tambahan, ini, hanya, berjalan, setahun, yakni, pada, 1999, .], [Warga, setempat, mengaku, tak, pernah, lagi, menerima, makanan, tambahan, bagi, anak-anak, kurang, gizi, sejak, dua, tahun, terakhir, ., (, TNA/Bisri, Merduani, ), .]]","[[Sebanyak, 51, anak, di, bawah, usia, lima, tahun, di, Kota, Madya, Bandar, Lampung, ,, menderita, busung, lapar, karena, kekurangan, gizi, .], [Lima, di, antaranya, meninggal, dunia, .]]","[0, 1]"
16030.json,"[[Liputan6, ., com, ,, Jakarta, :, Polemik, seputar, pelaksanaan, Sidang, Istimewa, MPR, ,, 1, Agustus, mendatang, ,, terus, bergulir, .], [Setelah, sekian, kali, ,, Presiden, Abdurrahman, Wahid, kembali, mengeluarkan, ancaman, akan, memberlakukan, status, keadaan, bahaya, dan, darurat, sipil, .], [Langkah, itu, bakal, dilakukan, jika, SI, MPR, tetap, meminta, pertanggungjawaban, dirinya, .], [Penegasan, tersebut, diungkapkan, Presiden, Wahid, dalam, dialog, usai, salat, Jumat, di, 

- Canonical Train Dataframe

In [14]:
df_canonical_train = pd.DataFrame.from_dict(data_canonical_train, orient="index")
df_canonical_train = df_canonical_train.drop(columns=['url', 'id'])
df_canonical_train.head()

,clean_article,clean_summary,extractive_summary
228132.json,"[[TIGA, kali, somasi, dilayangkan, kepada, Nuri, Shaden, .], [Namun, reaksi, yang, diharapkan, agar, meminta, maaf, hasilnya, nihil, .], [Keluarga, Janu, Utomo, melaporkan, kasus, kecelakaan, di, Jalan, Sisingamaraja, ,, Jakarta, Selatan, ,, 1, Juni, silam, itu, ke, Kepolisian, Resor, Metro, Jaksel, .], ["", Kita, melaporkan, atas, dugaan, tindak, pidana, ,, "", tutur, Taufik, Basari, ,, kuasa, hukum, keluarga, Janu, ,, belum, lama, ini, .], [Taufik, menambahkan, ,, pihaknya, ingin, Nuri, mencabut, pernyataan, di, depan, pers, ,, 2, Juni, silam, .], [Ia, juga, ingin, Nuri, meminta, maaf, di, media, atas, kejadian, itu, .], [Dan, juga, menyampaikan, belasungkawa, .], [Taufik, menilai, ,, pernyataan, Nuri, memutarbalikkan, fakta, .], ["", Yang, kita, adukan, soal, pernyataan-pernyataannya, ,, "", ujar, dia, .], [Keluarga, Janu, melaporkan, dengan, dua, tuduhan, .], [Yakni, sikap, yang, tidak, menyenangkan, dan, pencemaran, nama, baik, .], [Kala, itu, Nuri, menuding, mobil, ambulans, yang, menabrak, kendaraannya, .], ["", Intinya, mereka, menabrak, kita, ,, "", ucap, Nuri, saat, jumpa, pers, waktu, itu, .], [Pernyataan, disampaikan, bintang, film, Seventeen, didampingi, Noni, T, ,, pengacaranya, .], [Ia, menyalahkan, Januari, Purwoko, ,, sopir, ambulans, .], [Padahal, Januari, sudah, mengklakson, mobil, Honda, Jazz, yang, ditumpangi, Nuri, .], [Tabrakan, pun, terhindarkan, .], [Ambulans, rusak, di, bagian, kanan, dan, as, roda, belakang, patah, .], [Tabrakan, mobil, ambulans, dan, Honda, Jazz, yang, dikendarai, Nuri, merenggut, nyawa, Janu, Utomo, .], [Sedangkan, Januari, Purwoko, ,, sopir, ambulans, cedera, parah, .], [Tiga, lainnya, di, mobil, ambulans, luka, ringan, .], [Yakni, istri, Janu, ,, Retno, Indarti, ,, Krisanti, Indriani, ,, dan, perawat, Risa, Citra, Dewi, .], [Kala, itu, keluarga, mengantarkan, Janu, hendak, mencuci, darah, .], [Soal, ini, kubu, Nuri, membantah, .], [Noni, mengatakan, Janu, meninggal, karena, penyakit, jantung, yang, dideritanya, ., (, As, ), .]]","[[TIGA, kali, somasi, dilayangkan, kepada, Nuri, Shaden, .], [Namun, reaksi, yang, diharapkan, agar, meminta, maaf, hasilnya, nihil, .], [Keluarga, Janu, Utomo, pun, melaporkan, Nuri, ke, Kepolisian, Resor, Metro, Jakarta, Selatan, .]]","[0, 1, 2]"
116936.json,"[[Liputan6, ., com, ,, Jakarta, :, Earth, Wind, and, Fire, Experience, Worldwide, Tour, 2006, Featuring, Al, Mckay, All, Star, sukses, memuaskan, penikmat, jazz, yang, memadati, Grand, Melia, ,, Jakarta, Selatan, ,, Rabu, (, 1/2, ), malam, .], [Pertunjukan, ini, adalah, penampilan, kedua, dari, rangkaian, tur, grup, asal, Chicago, ,, Amerika, Serikat, itu, di, enam, kota, besar, di, Indonesia, .], [Setelah, Jakarta, dan, Bandung, ,, mereka, siap, menggoyang, Semarang, ,, Yogyakarta, ,, Surabaya, ,, dan, Medan, .], [Tembang, Serpentine, Fire, menjadi, lagu, pembuka, Earth, Wind, and, Fire, .], [Dengan, energik, ,, trio, vokal, Tim, Owens, ,, Claude, Woods, ,, dan, Devere, Duckett, berhasil, memukau, penonton, .], [Diiringi, Al, Mckay, All, Star, ,, mereka, terus, mendendangkan, hits, andalan, ,, antara, lain, Pride, dan, Gotta, Get, You, Into, My, Life, .], [Meski, Earth, Wind, and, Fire, terbilang, grup, lawas, ,, tembang, mereka, tetap, digemari, .], [Buktinya, 1, ., 500, tiket, yang, disediakan, untuk, Jakarta, ludes, terjual, ., (, KEN/Teguh, Dwi, Hartono, ), .]]","[[Setelah, di, Bandung, ,, Earth, Wind, and, Fire, giliran, menghibur, penikmat, jazz, di, Jakarta, ,, tadi, malam, .], [Lagu-lagu, andalan, grup, asal, Chicago, itu, mampu, menyihir, penonton, sampai, detik, terakhir, pertunjukan, .]]","[1, 6]"
57896.json,"[[Liputan6, ., com, ,, Jakarta, :, Belasan, unit, mobil, pemadam, kebakaran, dikerahkan, untuk, menjinakkan, amukan, si, jago, merah, di, sebuah, rumah, di, kawasan, Perumahan, Patra, Kuningan, ,, Jakarta, Selatan, ,, Ahad, (, 6/7, ), sore, .], [Petugas, pemadam, juga, membuka, paksa, bagian, tengah, rumah, bernomor, sem

- Count Rows

In [15]:
row_canonical_dev = df_canonical_dev.shape[0]
row_canonical_test = df_canonical_test.shape[0]
row_canonical_train = df_canonical_train.shape[0]

print("Max rows canonical dev: ", row_canonical_dev)
print("Max rows canonical test: ", row_canonical_test)
print("Max rows canonical train: ", row_canonical_train)

Max rows canonical dev:  10972
Max rows canonical test:  10972
Max rows canonical train:  193883


## Convert to string

In [16]:
df_canonical_dev['clean_article'] = df_canonical_dev['clean_article'].apply(lambda x: ' '.join(' '.join(word) for word in x))
df_canonical_dev['clean_summary'] = df_canonical_dev['clean_summary'].apply(lambda x: ' '.join(' '.join(word) for word in x))
df_canonical_dev.head()

,clean_article,clean_summary,extractive_summary
3721.json,"Liputan6 . com , Jakarta : Keinginan untuk menindaklanjuti hasil curah pendapat yang diprakarsai Kwik Kian Gie , semakin mengental . Buktinya , beberapa anggota DPR ngebet membentuk Forum Lobi . Ketua DPR Akbar Tandjung baru-baru ini , mempersilakan para wakil rakyat yang ingin membentuk forum tersebut . Namun , Akbar menegaskan , pembentukan forum tersebut bukan bagian dari mekanisme DPR . Menurut Akbar , hasil curah pendapat 11 November silam , bisa saja dimasukkan dalam agenda dewan . Asalkan , kata Ketua Umum Golkar ini , semua proses harus berjalan sesuai prosedur yang ada . Artinya , itu harus disepakati melalui rapat fraksi dan dibawa ke Badan Musyawarah . "" Bila Bamus telah merumuskannya menjadi sebuah usulan agenda , maka dapat dibawa ke rapat paripurna yang nantinya akan menjadi agenda DPR , "" kata Akbar . ( YYT/Diah Kusuma dan Prihandoyo ) .","Sebagian anggota DPR berniat besar menindaklanjuti hasil curah pendapat menjadi agenda dewan . Akbar Tandjung setuju membentuk Forum Lobi , asal , sesuai prosedur yang ada di DPR .","[1, 4]"
2833.json,"Liputan6 . com , Jakarta : Gubernur DKI Jakarta Sutiyoso mengakui , pendapatan pajak dari kendaraan bermotor khususnya mobil mewah tidak didapat dengan optimal . Guna meningkatkan Pendapatan Asli Daerah DKI Jakarta , sutiyoso meminta Kepala Dinas Pendapatan Daerah menertibkan penerimaan pajak mobil mewah yang selama ini dinilai tak transparan . Penegasan ini disampaikannya seusai meresmikan Gedung Kantor Bersama Samsat Unit Pelayanan Pajak Kendaraan Bermotor dan Bea Balik Nama Kendaraan Bermotor di Jakarta Barat , Jumat ( 27/10 ) . Pajak mobil mewah , kata Sutiyoso , seharusnya dapat dioptimalkan seiring dengan banyaknya mobil mewah yang masuk ke Indonesia . Selama ini pemilik mobil mewah membayar pajaknya melalui oknum dengan biaya lebih murah . Karena itu , Pemerintah Daerah Jakarta akan bekerja sama dengan Kepolisian Daerah Metro Jaya untuk menangani sejumlah oknum yang membiarkan pemilik mobil mewah tidak membayar pajak . Menurut Sutiyoso , pajak kendaraan bermotor adalah andalan utama PAD DKI Jakarta . Pemilik mobil mewah sudah sepantasnya dikenakan pajak lebih mahal karena mereka tergolong orang kaya . ( ULF/Insan Kamil dan Satya Pandia ) .","Pendapatan Asli Daerah Jakarta kerap terkumpul secara tak optimal . Penyebabnya , banyak pemilik mobil mewah yang membayar pajak lewat calo .","[1, 4]"
729.json,"Liputan6 . com , Jakarta : Sejumlah mobil yang terjebak di ruang basement Gedung Bursa Efek Jakarta akibat ledakan bom di lantai parkir , Jumat ( 15/9 ) pagi , mulai diambil pemiliknya . Sebagian mobil lainnya belum bisa dikeluarkan karena masih harus diperiksa tim dari Pusat Laboratorium Forensik Kepolisian Republik Indonesia . Dari pantauan SCTV , sekitar pukul 10 . 00 WIB , jumlah mobil yang sudah dikeluarkan mencapai 45 unit . Untuk mengambil mobil tersebut , para pemilik harus menunjukkan bukti kepemilikan , seperti Surat Tanda Nomor Kendaraan Bermotor , Buku Pemilik Kendaraan Bermotor , dan karcis parkir . Mobil yang boleh diambil adalah mobil yang berada di luar garis polisi . Sedangkan mobil yang berada di dalam garis polisi belum bisa diambil seraya menunggu izin dari Tim Puslabfor Polri . Hal itu dilakukan agar penyelidikan polisi tak mengalami gangguan . Menurut petugas parkir , jumlah total mobil di lantai P2 saat terjadi ledakan mencapai 246 unit . Dari jumlah itu hanya 73 unit mobil yang dalam kondisi baik . Sementara itu , empat anggota Dewan Perwakilan Rakyat RI dari Fraksi Partai Demokrasi Indonesia-Perjuangan mengunjungi BEJ . Menurut Sukowaluyo Mintoraharjo , kasus peledakan ini harus secepatnya dituntaskan . Ini dikarenakan bursa efek merupakan institusi penting , yaitu sebagai salah satu simbol stabilitas perekonomian nasional . Hingga sekitar pukul 11 . 00 WIB puluhan karyawan yang berkantor di Gedung BEJ terlihat duduk-duduk di lobi . Mereka tampak membicarakan insiden yang telah menelan korba

In [17]:
df_canonical_test['clean_article'] = df_canonical_test['clean_article'].apply(lambda x: ' '.join(' '.join(word) for word in x))
df_canonical_test['clean_summary'] = df_canonical_test['clean_summary'].apply(lambda x: ' '.join(' '.join(word) for word in x))
df_canonical_test.head()

,clean_article,clean_summary,extractive_summary
23684.json,"Liputan6 . com , Bangka : Kapal patroli Angkatan Laut Republik Indonesia , Belinyu , baru-baru ini , menangkap tiga kapal nelayan berbendera Thailand , yakni KM Binatama , KM Sumber Jaya II , dan KM Mataram di Perairan Belitung Utara . Ketiga kapal itu ditangkap karena melanggar zona ekonomi ekslusif Indonesia . Saat ini , kapal-kapal itu diamankan di Pos Lanal Pelabuhan Pangkalan Balam , Bangka-Belitung . Menurut Komandan Pangkalan TNI AL Bangka Letnan Kolonel Laut Fredy Egam , selain menangkap tiga kapal , ALRI juga memeriksa 43 anak buah kapal . Mereka disergap saat sedang mengangkat jaring pukat harimau di Perairan Belitung Utara . Dari jumlah itu , hanya enam orang yang dijadikan tersangka , yakni tiga nahkoda dan tiga kepala kamar mesin kapal . Sedangkan ABK yang lain akan dideportasi ke negara asalnya . Meski berhasil menahan enam tersangka , TNI AL gagal mengamankan ikan tangkapan nelayan Thailand tersebut . Sebab , sebelum patroli datang , mereka telah memindahkan puluhan ton ikan hasil jaringan ke kapal induk . ( ULF/Ajmal Rokian dan Yanuar Ichrom ) .","Meski memiliki izin resmi , TNI AL tetap menangkap tiga kapan nelayan berbendera Thailand . Pasalnya , ketiga kapal itu melanggar zona ekonomi ekslusif dan menjaring ikan dengan pukat harimau .","[1, 7]"
24141.json,"Liputan6 . com , Bandar Lampung : Sebanyak 51 anak di bawah umur lima tahun terserang busung lapar atau marasmus karena kekurangan gizi di Kota Madya Bandar Lampung . Lima di antaranya tewas . Data tersebut diungkapkan Kepala Dinas Kesehatan Kota Bandar Lampung M . Sudarman , baru-baru ini . Menurut Sudarman , Dinas Kesehatan Bandar Lampung mencatat sekitar 51 anak terserang busung lapar yang tersebar di beberapa kecamatan , selama periode 1999 sampai 2001 . Kebanyakan anak penderita busung tersebut berasal dari keluarga yang hidup di bawah garis kemiskinan . Selain kekurangan gizi , komplikasi radang paru-paru juga menjadi satu faktor penyebab kematian anak penderita busung lapar tersebut . Data Dinas Kesehatan menunjukkan pada 1999 , ditemukan 41 anak terserang penyakit busung lapar . Sebagian besar penderita berdomisili di kampung miskin Umbul Kunci . Jumlah penderita busung lapar menurun pada 2000 , yakni hanya sembilan anak . Sedangkan September 2001 , seorang anak meninggal karena marasmus . Sudarman menegaskan , untuk menekan jumlah korban marasmus , anak-anak dan balita diberi makanan tambahan ke sentra-sentra rawan busung lapar seperti di Desa Umbul Unci . Tetapi , berdasarkan keterangan masyarakat Umbul Kunci , program makanan tambahan ini hanya berjalan setahun yakni pada 1999 . Warga setempat mengaku tak pernah lagi menerima makanan tambahan bagi anak-anak kurang gizi sejak dua tahun terakhir . ( TNA/Bisri Merduani ) .","Sebanyak 51 anak di bawah usia lima tahun di Kota Madya Bandar Lampung , menderita busung lapar karena kekurangan gizi . Lima di antaranya meninggal dunia .","[0, 1]"
16030.json,"Liputan6 . com , Jakarta : Polemik seputar pelaksanaan Sidang Istimewa MPR , 1 Agustus mendatang , terus bergulir . Setelah sekian kali , Presiden Abdurrahman Wahid kembali mengeluarkan ancaman akan memberlakukan status keadaan bahaya dan darurat sipil . Langkah itu bakal dilakukan jika SI MPR tetap meminta pertanggungjawaban dirinya . Penegasan tersebut diungkapkan Presiden Wahid dalam dialog usai salat Jumat di Mesjid Al Munawarah , Ciganjur , Jakarta Selatan , Jumat ( 06/7 ) . Bagi Gus Dur , pada dasarnya dia tidak mempermasalahkan pelaksanaan SI MPR . Asalkan , agenda yang dibahas tidak menyangkut pertanggungjawaban dan kinerja pemerintahannya . "" Sebab , masih ada agenda lain yang bisa dibahas , demi kemajuan negara ini , "" kata Gus Dur , serius . Pernyataan Gus Dur kontan ditanggap Wakil Rakyat . Menurut anggota Fraksi Partai Bulan Bintang DPR Hamdan Zoelva , pemberlakuan keadaan darurat sipil justru akan mempermudah aparat keamanan untuk mengantisipasi . Jadi , siapa pun yang berniat 

In [18]:
df_canonical_train['clean_article'] = df_canonical_train['clean_article'].apply(lambda x: ' '.join(' '.join(word) for word in x))
df_canonical_train['clean_summary'] = df_canonical_train['clean_summary'].apply(lambda x: ' '.join(' '.join(word) for word in x))
df_canonical_train.head()

,clean_article,clean_summary,extractive_summary
228132.json,"TIGA kali somasi dilayangkan kepada Nuri Shaden . Namun reaksi yang diharapkan agar meminta maaf hasilnya nihil . Keluarga Janu Utomo melaporkan kasus kecelakaan di Jalan Sisingamaraja , Jakarta Selatan , 1 Juni silam itu ke Kepolisian Resor Metro Jaksel . "" Kita melaporkan atas dugaan tindak pidana , "" tutur Taufik Basari , kuasa hukum keluarga Janu , belum lama ini . Taufik menambahkan , pihaknya ingin Nuri mencabut pernyataan di depan pers , 2 Juni silam . Ia juga ingin Nuri meminta maaf di media atas kejadian itu . Dan juga menyampaikan belasungkawa . Taufik menilai , pernyataan Nuri memutarbalikkan fakta . "" Yang kita adukan soal pernyataan-pernyataannya , "" ujar dia . Keluarga Janu melaporkan dengan dua tuduhan . Yakni sikap yang tidak menyenangkan dan pencemaran nama baik . Kala itu Nuri menuding mobil ambulans yang menabrak kendaraannya . "" Intinya mereka menabrak kita , "" ucap Nuri saat jumpa pers waktu itu . Pernyataan disampaikan bintang film Seventeen didampingi Noni T , pengacaranya . Ia menyalahkan Januari Purwoko , sopir ambulans . Padahal Januari sudah mengklakson mobil Honda Jazz yang ditumpangi Nuri . Tabrakan pun terhindarkan . Ambulans rusak di bagian kanan dan as roda belakang patah . Tabrakan mobil ambulans dan Honda Jazz yang dikendarai Nuri merenggut nyawa Janu Utomo . Sedangkan Januari Purwoko , sopir ambulans cedera parah . Tiga lainnya di mobil ambulans luka ringan . Yakni istri Janu , Retno Indarti , Krisanti Indriani , dan perawat Risa Citra Dewi . Kala itu keluarga mengantarkan Janu hendak mencuci darah . Soal ini kubu Nuri membantah . Noni mengatakan Janu meninggal karena penyakit jantung yang dideritanya . ( As ) .",TIGA kali somasi dilayangkan kepada Nuri Shaden . Namun reaksi yang diharapkan agar meminta maaf hasilnya nihil . Keluarga Janu Utomo pun melaporkan Nuri ke Kepolisian Resor Metro Jakarta Selatan .,"[0, 1, 2]"
116936.json,"Liputan6 . com , Jakarta : Earth Wind and Fire Experience Worldwide Tour 2006 Featuring Al Mckay All Star sukses memuaskan penikmat jazz yang memadati Grand Melia , Jakarta Selatan , Rabu ( 1/2 ) malam . Pertunjukan ini adalah penampilan kedua dari rangkaian tur grup asal Chicago , Amerika Serikat itu di enam kota besar di Indonesia . Setelah Jakarta dan Bandung , mereka siap menggoyang Semarang , Yogyakarta , Surabaya , dan Medan . Tembang Serpentine Fire menjadi lagu pembuka Earth Wind and Fire . Dengan energik , trio vokal Tim Owens , Claude Woods , dan Devere Duckett berhasil memukau penonton . Diiringi Al Mckay All Star , mereka terus mendendangkan hits andalan , antara lain Pride dan Gotta Get You Into My Life . Meski Earth Wind and Fire terbilang grup lawas , tembang mereka tetap digemari . Buktinya 1 . 500 tiket yang disediakan untuk Jakarta ludes terjual . ( KEN/Teguh Dwi Hartono ) .","Setelah di Bandung , Earth Wind and Fire giliran menghibur penikmat jazz di Jakarta , tadi malam . Lagu-lagu andalan grup asal Chicago itu mampu menyihir penonton sampai detik terakhir pertunjukan .","[1, 6]"
57896.json,"Liputan6 . com , Jakarta : Belasan unit mobil pemadam kebakaran dikerahkan untuk menjinakkan amukan si jago merah di sebuah rumah di kawasan Perumahan Patra Kuningan , Jakarta Selatan , Ahad ( 6/7 ) sore . Petugas pemadam juga membuka paksa bagian tengah rumah bernomor sembilan itu untuk menyelamatkan isi hunian di Kompleks Perumahan Pertamina tersebut . Puluhan petugas pemadam disebar ke beberapa tempat untuk mencegah meluasnya kobaran api yang mulai menyala sekitar pukul 17 . 00 WIB . Beberapa warga sekitar mengatakan , bangunan yang selama ini berfungsi sebagai tempat istirahat petugas kebersihan itu , pada malam hari biasanya dijadikan tempat mangkal sejumlah pemuda . Api di lokasi kebakaran yang berjarak beberapa puluh meter dari rumah mantan Presiden B . J . Habibie itu berhasil dipadamkan dalam tempo dua jam . Hingga kini , belum diketahui jumlah kerugian . Sehari sebelumnya , si jago 

## Save to CSV

In [84]:
df_canonical_dev.to_csv('canonical_dev.csv', index=False)
df_canonical_test.to_csv('canonical_test.csv', index=False)
df_canonical_train.to_csv('canonical_train.csv', index=False)

## Tokenize and Preprocess

In [39]:
val_data = Dataset.from_dict(df_canonical_dev)

- Di sini saya hanya menggunakan 10% data test karena keterbatasan hardware dan waktu

In [74]:
partial_test_data = df_canonical_test.sample(frac=0.1, random_state=42)
print("Jumlah data untuk training:", len(partial_test_data))

Jumlah data untuk training: 1097


In [75]:
test_data = Dataset.from_dict(partial_test_data)

In [76]:
print("Jumlah data untuk training:", len(test_data))

Jumlah data untuk training: 1097


- Di sini saya hanya menggunakan 10% data training karena keterbatasan hardware dan waktu

In [43]:
partial_train_data = df_canonical_train.sample(frac=0.1, random_state=42)
print("Jumlah data untuk training:", len(partial_train_data))

Jumlah data untuk training: 19388


In [44]:
train_data = Dataset.from_dict(partial_train_data)

In [45]:
print("Jumlah data untuk training:", len(train_data))

Jumlah data untuk training: 19388


In [46]:
# Tokenizer
from transformers import EncoderDecoderModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

In [47]:
tokenizer.bos_token, tokenizer.eos_token, tokenizer.get_vocab()[tokenizer.bos_token], tokenizer.get_vocab()[tokenizer.eos_token]

('[CLS]', '[SEP]', 3, 1)

In [48]:
#Define function to preprocess

prefix = tokenizer.bos_token + " "
posfix = " " + tokenizer.eos_token

def preprocess_function(examples):
    inputs = [prefix + str(doc) + posfix for doc in examples["clean_article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text=examples["clean_summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [49]:
tokenized_canonical_dev = val_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/10972 [00:00<?, ? examples/s]

In [77]:
tokenized_canonical_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/1097 [00:00<?, ? examples/s]

In [51]:
tokenized_canonical_train = train_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/19388 [00:00<?, ? examples/s]

In [52]:
#Set Bertconfig and EncoderDecoder Model

from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel

config_encoder = BertConfig(
    vocab_size = len(tokenizer.get_vocab()),
    hidden_size = 16,
    num_hidden_layers = 1,
    num_attention_heads = 2,
    intermediate_size = 32,
)
config_decoder = BertConfig(
    vocab_size = len(tokenizer.get_vocab()),
    hidden_size = 16,
    num_hidden_layers = 1,
    num_attention_heads = 2,
    intermediate_size = 32,
)

config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
model = EncoderDecoderModel(config=config)
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [53]:
# Set data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='cahya/t5-base-indonesian-summarization-cased') #t5-small

In [54]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-4, #2e-5
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=False,
    
)

In [55]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_canonical_train,
    eval_dataset=tokenized_canonical_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [56]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,9.427400,9.258271


Removed shared tensor {'decoder.cls.predictions.decoder.weight', 'decoder.cls.predictions.decoder.bias'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading
/Users/tatauntoro/anaconda3/lib/python3.11/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:619: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/Users/tatauntoro/anaconda3/lib/python3.11/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:639: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuni

TrainOutput(global_step=1212, training_loss=9.620941867135933, metrics={'train_runtime': 2308.4668, 'train_samples_per_second': 8.399, 'train_steps_per_second': 0.525, 'total_flos': 2167087762944.0, 'train_loss': 9.620941867135933, 'epoch': 1.0})

### Result

Training Loss:
- 9.427400

Validation Loss:
- 9.258271

Notes:
- Training was done by 10% training and test data. It's because hardware issue 

## Summarization

In [63]:
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained("cahya/bert2gpt-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2gpt-indonesian-summarization")

# 
ARTICLE_TO_SUMMARIZE = "Liputan6 . com , Jakarta : Gubernur DKI Jakarta Sutiyoso mengakui , pendapatan pajak dari kendaraan bermotor khususnya mobil mewah tidak didapat dengan optimal . Guna meningkatkan Pendapatan Asli Daerah DKI Jakarta , sutiyoso meminta Kepala Dinas Pendapatan Daerah menertibkan penerimaan pajak mobil mewah yang selama ini dinilai tak transparan . Penegasan ini disampaikannya seusai meresmikan Gedung Kantor Bersama Samsat Unit Pelayanan Pajak Kendaraan Bermotor dan Bea Balik Nama Kendaraan Bermotor di Jakarta Barat , Jumat ( 27/10 ) . Pajak mobil mewah , kata Sutiyoso , seharusnya dapat dioptimalkan seiring dengan banyaknya mobil mewah yang masuk ke Indonesia . Selama ini pemilik mobil mewah membayar pajaknya melalui oknum dengan biaya lebih murah . Karena itu , Pemerintah Daerah Jakarta akan bekerja sama dengan Kepolisian Daerah Metro Jaya untuk menangani sejumlah oknum yang membiarkan pemilik mobil mewah tidak membayar pajak . Menurut Sutiyoso , pajak kendaraan bermotor adalah andalan utama PAD DKI Jakarta . Pemilik mobil mewah sudah sepantasnya dikenakan pajak lebih mahal karena mereka tergolong orang kaya . ( ULF/Insan Kamil dan Satya Pandia ) ."

# generate summary
input_ids = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors='pt')
summary_ids = model.generate(input_ids,
            min_length=20,
            max_length=80, 
            num_beams=10,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.8,
            top_k = 50,
            top_p = 0.95)

summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.26k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

/Users/tatauntoro/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


gubernur dki jakarta sutiyoso mengakui pendapatan pajak dari kendaraan bermotor khususnya mobil mewah tidak didapat dengan optimal. pemda jakarta akan bekerja sama dengan polda metro jaya untuk menangani sejumlah oknum yang membiarkan pemilik mobil


In [65]:
article = """
    Liputan6 . com , Jakarta : Keinginan untuk menindaklanjuti hasil curah pendapat yang diprakarsai Kwik Kian Gie , semakin mengental . Buktinya , beberapa anggota DPR ngebet membentuk Forum Lobi . Ketua DPR Akbar Tandjung baru-baru ini , mempersilakan para wakil rakyat yang ingin membentuk forum tersebut . Namun , Akbar menegaskan , pembentukan forum tersebut bukan bagian dari mekanisme DPR . Menurut Akbar , hasil curah pendapat 11 November silam , bisa saja dimasukkan dalam agenda dewan . Asalkan , kata Ketua Umum Golkar ini , semua proses harus berjalan sesuai prosedur yang ada . Artinya , itu harus disepakati melalui rapat fraksi dan dibawa ke Badan Musyawarah . " Bila Bamus telah merumuskannya menjadi sebuah usulan agenda , maka dapat dibawa ke rapat paripurna yang nantinya akan menjadi agenda DPR , " kata Akbar . ( YYT/Diah Kusuma dan Prihandoyo ) .
"""
summary = """
    Sebagian anggota DPR berniat besar menindaklanjuti hasil curah pendapat menjadi agenda dewan . Akbar Tandjung setuju membentuk Forum Lobi , asal , sesuai prosedur yang ada di DPR
"""

In [66]:
def predict_summary(document):
  device = model.device
  tokenized = tokenizer([document], max_length=512, truncation =True, padding ='longest',return_tensors='pt')
  tokenized = {k: v.to(device) for k, v in tokenized.items()}
  tokenized_result = model.generate(**tokenized, max_length=128)
  tokenized_result = tokenized_result.to('cpu')
  predicted_summary = tokenizer.decode(tokenized_result[0])
  return predicted_summary

In [67]:
predicted_summary = predict_summary(article)

In [68]:
predicted_summary

'[CLS] keinginan untuk menindaklanjuti hasil curah pendapat yang diprakarsai kwik kian gie, semakin mengental. ketua dpr akbar tandjung mempersilakan para wakil rakyat yang ingin membentuk forum lobi [SEP]'

In [69]:
summary

'\n    Sebagian anggota DPR berniat besar menindaklanjuti hasil curah pendapat menjadi agenda dewan . Akbar Tandjung setuju membentuk Forum Lobi , asal , sesuai prosedur yang ada di DPR\n'

## Evaluation

In [78]:
from rouge import Rouge

def get_rouge_scores(actual_summary, predicted_summary):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, actual_summary)
    return [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']]

In [79]:
# rouge score of validation data

from tqdm import tqdm
import pandas as pd

rouge1_scores = []
rouge2_scores = []
rougel_scores = []

pred_summary_list = []

for i in tqdm(range(len(tokenized_canonical_test['clean_summary']))):

  doc = tokenized_canonical_test['clean_article'][i]
  pred_summary = predict_summary(doc)
  human_summary = tokenized_canonical_test['clean_summary'][i]

  score = get_rouge_scores(human_summary, pred_summary)

  rouge1_scores.append(score[0])
  rouge2_scores.append(score[1])
  rougel_scores.append(score[2])

  pred_summary_list.append(pred_summary)

eval = pd.DataFrame({
    "pred_summary" : pred_summary_list,
    "rouge1" : rouge1_scores,
    "rouge2" : rouge2_scores,
    "rougel" : rougel_scores,
})


100%|█████████████████████████████████████████████████████████████████████| 1097/1097 [1:50:46<00:00,  6.06s/it]


In [80]:
# average rouge 1
eval['rouge1'].mean()

0.24273178594926

In [81]:
# average rouge 2
eval['rouge2'].mean()

0.10311634728285245

In [82]:
# average rouge l
eval['rougel'].mean()

0.2275334810607534

In [83]:
eval

,pred_summary,rouge1,rouge2,rougel
0,"[CLS] pengamat politik dewi fortuna anwar mengatakan, ada empat faktor penyebab jatuhnya gus dur. ia juga tak mendapat dukungan dari militer. gus dur juga dianggap kalah dalam pertarungan antarelite politik. [SEP]",0.120000,0.000000,0.120000
1,[CLS] mantan presiden abdurrahman wahid menilai kabinet gotong royong masih diisi orang - orang bermasalah yang seharusnya tidak berada di kabinet. beberapa menteri masih terlibat persoalan hukum. [SEP],0.208333,0.080000,0.208333
2,"[CLS] bank muamalat berencana meningkatkan kredit sebesar rp 1 triliun atau naik 63 persen dibanding tahun 1999. manajemen tak akan menurunkan jumlah kredit, hanya berupaya menambah modal sebesar rp 200 miliar. [SEP]",0.350877,0.070175,0.315789
3,[CLS] secara keseluruhan lawatan presiden megawati sukarnoputri ke amerika serikat dan jepang mendapat dukungan untuk menyelesaikan masalah pembangunan dan persoalan - persoalan yang terjadi di indonesia. [SEP],0.212766,0.040000,0.170213
4,[CLS] presiden abdurrahman wahid akan menunjuk marsilam simanjuntak sebagai jaksa agung menggantikan lopa. sedangkan posisi mantan pejabat lbh dan hak asasi manusia akan digantikan menteri dalam negeri. [SEP],0.000000,0.000000,0.000000
...,...,...,...,...
1092,[CLS] banyak cara mengungkapkan rasa syukur atas anugerah sang pencipta. salah satunya dengan menggelar balap burung merpati. merpati aduan juga memerlukan telur kampung dan jamu tradisional. [SEP],0.117647,0.000000,0.117647
1093,"[CLS] gedung land xiii, tempat mantan presiden sukarno diadili, hingga kini masih berdiri kokoh dengan konstruksi asli. semua bagian gedung tetap dipertahankan seperti ketika pertama kali dibangun. [SEP]",0.076923,0.000000,0.076923
1094,"[CLS] setelah menyala secara normal sekitar enam bulan, pln tanjungkarang, lampung kembali melakukan pemadaman aliran listrik secara bergiliran. langkah tersebut diprotes masyarakat setempat karena tidak ada pemberitahuan jadwal. [SEP]",0.326531,0.250000,0.326531
1095,"[CLS] setelah jajak pendapat di timtim selesai, untaet masih mempercayai pertamina untuk menyuplai bahan akar minyak ke sana. pertamina meraup keuntungan sekitar rp 4 miliar setiap bulan dengan menggunakan standar harga dolar amroz [SEP]",0.370370,0.181818,0.370370
